# Generación de embeddings

## Carga de librerías

In [1]:
from ast import literal_eval
from sentence_transformers import SentenceTransformer

import pandas as pd
import torch
import pickle

2025-09-09 16:33:52.932187: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757435633.299601      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757435633.406813      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Carga y transformación de datos

In [2]:
data_path = "../data/topics.csv"

In [5]:
df_topics = pd.read_csv(data_path)
df_topics.head().iloc[:, 1:]

,tid,n_messages,description,keywords,representative_docs,theme
0,0,111,0_vacunamos_novavax_fauci_reacciones,"['vacunamos', 'novavax', 'fauci', 'reacciones'...",['estracto de la agencia europea del medicamen...,Conspiracy theories and misinformation about C...
1,1,106,1_encontrado_oficialialistas_mujer_desconectado,"['encontrado', 'oficialialistas', 'mujer', 'de...","['los viajes espaciales, la creacion de una pa...",The theme revolves around conspiracy theories ...
2,2,85,2_expreso0demedianoche_directo_anonimo_descarg...,"['expreso0demedianoche', 'directo', 'anonimo',...",['el expreso de media noche directo del 24/02 ...,Unauthorized use of a popular podcast's name a...
3,3,65,3_despiertos_actuamos_inocencia_hijos,"['despiertos', 'actuamos', 'inocencia', 'hijos...",['da mucho asco ver ciertos personajes que van...,"Critique of egoism, superficiality, and lack o..."
4,4,63,4_protestas_republica_belga_cerrados,"['protestas', 'republica', 'belga', 'cerrados'...",['la gente se defiende: suiza fuerza un refere...,Public discontent and resistance against gover...


In [6]:
# Columna de strings a listas
df_topics["representative_docs"] = df_topics["representative_docs"].apply(literal_eval)

In [7]:
# Una fila por doc
cols = ["_id", "tid", "representative_docs"]
df_topics_expl = (df_topics[cols]
                  .explode("representative_docs", ignore_index=True))
df_topics_expl.head().iloc[:, 1:]

,tid,representative_docs
0,0,estracto de la agencia europea del medicamento...
1,0,""" si me vacunan: 1 . - ¿ puedo dejar de usar l..."
2,0,buenos días la ema avisa de que la segunda do...
3,1,"los viajes espaciales, la creacion de una pand..."
4,1,2011...este articulo del periodico sovereign i...


In [8]:
df_topics_expl.shape

(64842, 3)

In [9]:
# Evitar repetir el mismo doc dentro del mismo tópico en un mismo canal
df_topics_expl = (df_topics_expl
                  .drop_duplicates()
                  .reset_index(drop=True))

In [10]:
df_topics_expl.shape

(64228, 3)

In [11]:
# Nº de palabras en los mensajes
df_topics_expl["representative_docs"].apply(lambda x: len(x.split(" "))).describe()

count    64228.000000
mean       177.649156
std        172.629051
min         14.000000
25%         55.000000
50%        120.000000
75%        219.000000
max       1034.000000
Name: representative_docs, dtype: float64

## Generación de embeddings

In [12]:
# Inicialización del modelo
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2", device=device)
model.max_seq_length = 512

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/723 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/402 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
E = model.encode(df_topics_expl["representative_docs"], batch_size=512,
                 normalize_embeddings=True, show_progress_bar=True)

Batches:   0%|          | 0/126 [00:00<?, ?it/s]

In [14]:
E.shape

(64228, 768)

In [16]:
# Guardar los embeddings
with open("../data/embeddings.pkl", "wb") as f:
    pickle.dump(E, f)